In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def Q1_facility_member_charge(conn):
    """Query list of facilites to determine which charge fee to members"""
    cur = conn.cursor()

    query2="""SELECT name FROM Facilities WHERE membercost > 0.0"""

    cur.execute(query2)

    rows = cur.fetchall()

    print(rows)

def Q2_facility_no_member_charge(conn):
    """Query list of facilities to determine number that do not charge member fee"""
    cur= conn.cursor()

    query3="""SELECT COUNT(name) FROM Facilities WHERE membercost = 0.0"""

    cur.execute(query3)

    rows = cur.fetchall()

    print(rows)

def Q3_facility_member_charge_20(conn):
    """Query list of facilities to determine which charge fee to members,
    where fee is less than 20% of monthly maintenance cost"""
    cur = conn.cursor()

    query4= """SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost < monthlymaintenance * .20"""

    cur.execute(query4)

    rows = cur.fetchall()

    print(rows)

def Q4_facility_id(conn):
    """Retreive details of facilities with ID 1 and 5"""
    cur = conn.cursor()

    query5="""SELECT * FROM Facilities WHERE facid IN (1,5)"""

    cur.execute(query5)

    rows = cur.fetchall()

    print(rows)

def Q5_facility_cheap_expensive(conn):
    """Produce a list of facilities labeled 'cheap' or 'expensive' based on monthly
    maintenance cost"""
    cur = conn.cursor()

    query6="""SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100
    THEN 'expensive' ELSE 'cheap' END FROM Facilities"""

    cur.execute(query6)

    rows= cur.fetchall()

    print(rows)
    
def Q6_member_name(conn):
    """Get the first and last name of the last member(s) who signed up"""
    cur= conn.cursor()

    query7="""SELECT firstname, surname FROM Members WHERE joindate IN (SELECT MAX(joindate) FROM Members)"""

    cur.execute(query7)

    rows= cur.fetchall()

    print(rows)

def Q7_member_tennis(conn):
    """Produce a list of all members who have used a tennis court"""
    cur= conn.cursor()

    query8="""SELECT DISTINCT firstname ||' '|| surname AS membername,name
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    INNER JOIN Members M
    ON B.memid = M.memid
    WHERE name LIKE 'Tennis Court%'
    ORDER BY membername"""

    cur.execute(query8)

    rows= cur.fetchall()

    print(rows)

def Q8_bookings_2012_09_14(conn):
    """Produce a list of bookings on 2012-09-14 which will cost more than $30"""
    cur= conn.cursor()

    query9= """SELECT name, firstname ||' '|| surname AS membername, guestcost, membercost
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    INNER JOIN Members M
    ON B.memid = M.memid
    WHERE starttime LIKE '2012-09-14%'
    AND guestcost > 30.0 OR membercost > 30.0
    ORDER BY guestcost DESC"""

    cur.execute(query9)

    rows= cur.fetchall()

    print(rows)

def Q9_bookings_2012_09_14_sq(conn):
    """Produce a list of bookings on 2012-09-14 which will cost more than $30, using subqueries"""
    cur= conn.cursor()

    query10="""SELECT
		membername,
		facname,
		guestcost,
		membercost
FROM (
    	SELECT (SELECT firstname ||' '|| surname FROM Members M WHERE M.memid = B.memid) AS membername,
    			(SELECT name FROM Facilities F WHERE F.facid = B.facid) AS facname,
    			(SELECT guestcost FROM Facilities F WHERE F.facid = B.facid) AS guestcost,
    			(SELECT membercost FROM Facilities F WHERE F.facid = B.facid) AS membercost,
    			starttime AS date
    	FROM Bookings B
		WHERE starttime LIKE '2012-09-14%'
    	ORDER BY guestcost DESC) A
WHERE A.guestcost > 30 OR A.membercost > 30"""

    cur.execute(query10)

    rows= cur.fetchall()

    print(rows)

def Q10_facility_revenue(conn):
    """Produce a list of facilities with total revenue less than 1000"""
    cur= conn.cursor()

    query11="""SELECT name,revenue
    FROM (SELECT name,
    SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS revenue
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    GROUP BY name) 
    WHERE revenue < 1000
    ORDER BY revenue"""
    
    cur.execute(query11)

    rows= cur.fetchall()

    print(rows)

def Q11_recommended_members(conn):
    """Produce a report of member and those whoe recommended them"""
    cur= conn.cursor()

    query12="""SELECT M.surname ||','|| M.firstname AS member_name, R.surname ||','|| R.firstname AS recommended_by
    FROM Members M
    INNER JOIN Members R
    ON M.recommendedby = R.memid
    ORDER BY M.surname ||','|| M.firstname"""
    
    cur.execute(query12)

    rows= cur.fetchall()

    print(rows)

def Q12_facility_member_usage(conn):
    """Find the facilities usage by member"""
    cur= conn.cursor()

    query13="""SELECT name,usage_count
    FROM (SELECT name,
    COUNT(CASE WHEN memid > 0 THEN 1 * slots END) AS usage_count
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    GROUP BY name) 
    ORDER BY usage_count"""

    cur.execute(query13)

    rows= cur.fetchall()

    print(rows)

def Q13_facility_month_usage(conn):
    """Find the facilities usage by month"""
    cur= conn.cursor()

    query14="""SELECT name, usage_month, count
    FROM (SELECT name,
    strftime('%m',starttime) AS usage_month,
    COUNT(slots) AS count
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    GROUP BY name, strftime('%m',starttime)) 
    ORDER BY usage_month"""

    cur.execute(query14)

    rows=cur.fetchall()

    print(rows)
    
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        #print("2. Query all tasks")
        #select_all_tasks(conn)
        print('Question 1:')
        Q1_facility_member_charge(conn)
        
        print('Question 2:')
        Q2_facility_no_member_charge(conn)
        
        print('Question 3:')
        Q3_facility_member_charge_20(conn)
        
        print('Question 4:')
        Q4_facility_id(conn)

        print('Question 5:')
        Q5_facility_cheap_expensive(conn)

        print('Question 6:')
        Q6_member_name(conn)

        print('Question 7:')
        Q7_member_tennis(conn)

        print('Question 8:')
        Q8_bookings_2012_09_14(conn)

        print('Question 9:')
        Q9_bookings_2012_09_14_sq(conn)

        print('Question 10:')
        Q10_facility_revenue(conn)

        print('Question 11:')
        Q11_recommended_members(conn)

        print('Question 12:')
        Q12_facility_member_usage(conn)

        print('Question 13:')
        Q13_facility_month_usage(conn)
 
if __name__ == '__main__':
    main()


2.6.0
Question 1:
[('Tennis Court 1',), ('Tennis Court 2',), ('Massage Room 1',), ('Massage Room 2',), ('Squash Court',)]
Question 2:
[(4,)]
Question 3:
[(0, 'Tennis Court 1', 5, 200), (1, 'Tennis Court 2', 5, 200), (2, 'Badminton Court', 0, 50), (3, 'Table Tennis', 0, 10), (4, 'Massage Room 1', 9.9, 3000), (5, 'Massage Room 2', 9.9, 3000), (6, 'Squash Court', 3.5, 80), (7, 'Snooker Table', 0, 15), (8, 'Pool Table', 0, 15)]
Question 4:
[(1, 'Tennis Court 2', 5, 25, 8000, 200), (5, 'Massage Room 2', 9.9, 80, 4000, 3000)]
Question 5:
[('Tennis Court 1', 200, 'expensive'), ('Tennis Court 2', 200, 'expensive'), ('Badminton Court', 50, 'cheap'), ('Table Tennis', 10, 'cheap'), ('Massage Room 1', 3000, 'expensive'), ('Massage Room 2', 3000, 'expensive'), ('Squash Court', 80, 'cheap'), ('Snooker Table', 15, 'cheap'), ('Pool Table', 15, 'cheap')]
Question 6:
[('Darren', 'Smith')]
Question 7:
[('Anne Baker', 'Tennis Court 1'), ('Anne Baker', 'Tennis Court 2'), ('Burton Tracy', 'Tennis Court 2'),